In [1]:
# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Database
from sqlalchemy import create_engine

# (Optional) If using yfinance or APIs later
# import yfinance as yf


In [2]:
# MySQL connection details
DB_USER = "root"
DB_PASSWORD = "1433"
DB_HOST = "localhost"
DB_NAME = "financial_analytics"

engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)


In [3]:
# Load existing market prices from MySQL
query = """
SELECT asset_id, trade_date, adj_close, volume
FROM market_prices
"""

existing_df = pd.read_sql(query, engine)

existing_df["trade_date"] = pd.to_datetime(existing_df["trade_date"])
existing_df.head()


,asset_id,trade_date,adj_close,volume
0,11,2019-01-02,2510.030029,3733160000
1,11,2019-01-03,2447.889893,3858830000
2,11,2019-01-04,2531.939941,4234140000
3,11,2019-01-07,2549.689941,4133120000
4,11,2019-01-08,2574.409912,4120060000


In [4]:
# Simulate new weekly data
last_date = existing_df["trade_date"].max()

new_dates = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=5,
    freq="B"   # business days
)

new_data = []

for asset_id in existing_df["asset_id"].unique():
    last_price = existing_df.loc[
        existing_df["asset_id"] == asset_id, "adj_close"
    ].iloc[-1]

    for d in new_dates:
        price = last_price * (1 + np.random.normal(0, 0.01))
        volume = int(np.random.uniform(1e7, 5e7))

        new_data.append([asset_id, d, price, volume])

new_df = pd.DataFrame(
    new_data,
    columns=["asset_id", "trade_date", "adj_close", "volume"]
)

new_df.head()


,asset_id,trade_date,adj_close,volume
0,11,2026-01-29,6803.488587,37631923
1,11,2026-01-30,6909.288347,20834993
2,11,2026-02-02,6834.771020,16558762
3,11,2026-02-03,6756.417893,47952327
4,11,2026-02-04,6793.753751,42447618


In [5]:
# Remove duplicates
new_df.drop_duplicates(
    subset=["asset_id", "trade_date"],
    inplace=True
)

# Ensure positive prices
new_df = new_df[new_df["adj_close"] > 0]

# Add audit timestamp
new_df["created_at"] = datetime.now()

new_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   asset_id    55 non-null     int64         
 1   trade_date  55 non-null     datetime64[ns]
 2   adj_close   55 non-null     float64       
 3   volume      55 non-null     int64         
 4   created_at  55 non-null     datetime64[us]
dtypes: datetime64[ns](1), datetime64[us](1), float64(1), int64(2)
memory usage: 2.3 KB


In [6]:
# Append new data to MySQL
new_df.to_sql(
    "market_prices",
    engine,
    if_exists="append",
    index=False
)

print("MySQL market_prices table updated successfully")


MySQL market_prices table updated successfully


In [7]:
# Merge old + new for CSV
updated_df = pd.concat(
    [existing_df, new_df[existing_df.columns]],
    ignore_index=True
)

# Save to processed folder
csv_path = "../data/processed/clean_market_prices_week1.csv"

updated_df.to_csv(csv_path, index=False)

print("CSV updated for Power BI refresh")


CSV updated for Power BI refresh


In [8]:
with open("../data/processed/refresh_log.txt", "a") as f:
    f.write(f"Refresh successful at {datetime.now()}\n")
